<a href="https://colab.research.google.com/github/aderdouri/EiCNAM/blob/master/Tutorials/european_call_cva.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch

# Parameters for the European call option
S0 = 100.0  # Initial stock price
K = 90.0  # Strike price
T = 2.0  # Time to maturity (2 years)
r = 0.01  # Risk-free rate
sigma = 0.25  # Volatility

# CVA and intensity model parameters
LGD = 0.6  # Loss given default (e.g., 60%)
lambda_0 = 1.0  # Initial hazard rate
k = 0.5  # Mean-reversion speed
mu = 1.0  # Long-term mean of intensity
nu = 0.25  # Volatility of intensity
n_paths = 10000  # Number of Monte Carlo simulations
time_steps = 100  # Number of time steps in the simulation

# Generate time grid
dt = T / time_steps
time_grid = torch.linspace(0, T, time_steps)

# Simulate stock price paths using Geometric Brownian Motion (GBM)
# Simulate stock price paths using Geometric Brownian Motion (GBM)
def simulate_stock_paths(S0, r, sigma, T, time_steps, n_paths):
    dt = T / time_steps
    # Convert dt to a PyTorch tensor
    dt = torch.tensor(dt, dtype=torch.float32)
    Z = torch.randn(n_paths, time_steps)  # Random shocks
    increments = (r - 0.5 * sigma**2) * dt + sigma * torch.sqrt(dt) * Z
    increments = torch.cat([torch.zeros(n_paths, 1), increments], dim=1)  # Initial price
    log_S = torch.cumsum(increments, dim=1)
    S = S0 * torch.exp(log_S)
    return S

# Simulate default times using an intensity model
def simulate_default_times(lambda_0, time_steps, T, n_paths):
    dt = T / time_steps
    # Convert lambda_0 and dt to tensors
    lambda_0 = torch.tensor(lambda_0, dtype=torch.float32)
    dt = torch.tensor(dt, dtype=torch.float32)
    default_probs = 1 - torch.exp(-lambda_0 * dt)  # Default probabilities per step
    defaults = torch.rand(n_paths, time_steps) < default_probs  # Simulate defaults
    # Convert defaults to integer type before applying argmax
    default_times = torch.argmax(defaults.type(torch.int), dim=1) * dt  # First default time
    default_times[default_times == 0] = T + 1  # No default
    return default_times


# Calculate CVA
def calculate_cva(LGD, S, K, T, r, lambda_0, default_times, time_grid):
    # Compute option payoff at maturity
    payoffs = torch.maximum(S[:, -1] - K, torch.tensor(0.0))  # Max(S_T - K, 0)

    # Discount factor to maturity
    discount_factors = torch.exp(-torch.tensor(r, dtype=torch.float32) * torch.tensor(T, dtype=torch.float32)) # Convert r and T to tensors

    # Compute survival probabilities
    survival_probs = torch.exp(-lambda_0 * time_grid[-1])

    # Compute CVA
    cva = LGD * torch.mean((1 - survival_probs) * discount_factors * payoffs)
    return cva.item()

# Simulate stock prices and default times
S = simulate_stock_paths(S0, r, sigma, T, time_steps, n_paths)
default_times = simulate_default_times(lambda_0, time_steps, T, n_paths)

# Calculate CVA
cva = calculate_cva(LGD, S, K, T, r, lambda_0, default_times, time_grid)

print(f"Calculated CVA: {cva:.4f}")

In [ ]:
# Parameters
n_paths = 20000  # Reduced paths for memory efficiency
epsilon = 1e-5  # Small increment for lambda_0

# Simulate stock prices
S = simulate_stock_paths(S0, r, sigma, T, time_steps, n_paths)

# Calculate CVA for baseline lambda_0
default_times = simulate_default_times(lambda_0, time_steps, T, n_paths)
cva_base = calculate_cva(LGD, S, K, T, r, lambda_0, default_times, time_grid)

# Calculate CVA for slightly increased lambda_0
lambda_0_plus = lambda_0 + epsilon
default_times_plus = simulate_default_times(lambda_0_plus, time_steps, T, n_paths)
cva_plus = calculate_cva(LGD, S, K, T, r, lambda_0_plus, default_times_plus, time_grid)

# Delta of CVA with respect to lambda_0
delta_cva = (cva_plus - cva_base) / epsilon

print(f"CVA (baseline): {cva_base:.4f}")
print(f"CVA (lambda_0 + epsilon): {cva_plus:.4f}")
print(f"Delta of CVA w.r.t lambda_0: {delta_cva:.4f}")


In [ ]:
import torch

# Parameters for the European call option
S0 = 100.0  # Initial stock price
K = 90.0  # Strike price
T = 2.0  # Time to maturity (2 years)
r = 0.01  # Risk-free rate
sigma = 0.25  # Volatility

# CVA and intensity model parameters
LGD = 0.6  # Loss given default
lambda_0 = 1.0  # Initial hazard rate
epsilon = 1e-4  # Small increment for lambda_0
n_paths = 2000  # Number of Monte Carlo simulations
time_steps = 100  # Number of time steps in the simulation

# Generate time grid
dt = T / time_steps
time_grid = torch.linspace(0, T, time_steps)


# Simulate stock prices and default times
S = simulate_stock_paths(S0, r, sigma, T, time_steps, n_paths)
default_times = simulate_default_times(lambda_0, time_steps, T, n_paths)

# Calculate CVA for baseline lambda_0
cva_base = calculate_cva(LGD, S, K, T, r, lambda_0, default_times, time_grid)

# Calculate CVA for slightly increased lambda_0
lambda_0_plus = lambda_0 + epsilon
default_times_plus = simulate_default_times(lambda_0_plus, time_steps, T, n_paths)
cva_plus = calculate_cva(LGD, S, K, T, r, lambda_0_plus, default_times_plus, time_grid)

# Delta of CVA with respect to lambda_0
delta_cva = (cva_plus - cva_base) / epsilon

print(f"CVA (baseline): {cva_base:.5f}")
print(f"CVA (lambda_0 + epsilon): {cva_plus:.5f}")
print(f"Delta of CVA w.r.t lambda_0: {delta_cva:.5f}")


In [ ]:
import torch

# Parameters for the European call option
S0 = 100.0
K = 90.0
T = 2.0
r = 0.01
sigma = 0.25

# CVA and intensity model parameters
LGD = 0.6
lambda_0 = 1.0
epsilon = 1e-4  # Perturbation for lambda_0
n_paths = 500000  # Number of Monte Carlo paths
time_steps = 200  # Higher resolution for time steps

# Generate time grid
dt = T / time_steps
time_grid = torch.linspace(0, T, time_steps)

# Set random seed for reproducibility
torch.manual_seed(42)

# Simulate stock price paths using Geometric Brownian Motion (GBM)
def simulate_stock_paths(S0, r, sigma, T, time_steps, n_paths):
    dt = T / time_steps
    # Convert dt to a PyTorch tensor
    dt = torch.tensor(dt, dtype=torch.float32) # This line converts dt to a tensor
    Z = torch.randn(n_paths, time_steps)  # Random shocks
    increments = (r - 0.5 * sigma**2) * dt + sigma * torch.sqrt(dt) * Z
    increments = torch.cat([torch.zeros(n_paths, 1), increments], dim=1)
    log_S = torch.cumsum(increments, dim=1)
    S = S0 * torch.exp(log_S)
    return S

# Simulate default times using an intensity model
def simulate_default_times(lambda_0, time_steps, T, n_paths):
    dt = T / time_steps
    # Convert lambda_0 and dt to PyTorch tensors
    lambda_0 = torch.tensor(lambda_0, dtype=torch.float32)
    dt = torch.tensor(dt, dtype=torch.float32)
    default_probs = 1 - torch.exp(-lambda_0 * dt)
    defaults = torch.rand(n_paths, time_steps) < default_probs
    default_times = torch.argmax(defaults.type(torch.int), dim=1) * dt
    default_times[default_times == 0] = T + 1  # No default
    return default_times

# Calculate CVA
def calculate_cva(LGD, S, K, T, r, lambda_0, default_times, time_grid):
    payoffs = torch.maximum(S[:, -1] - K, torch.tensor(0.0))  # Max(S_T - K, 0)
    # Convert r and T to tensors before using them in torch.exp
    discount_factors = torch.exp(-torch.tensor(r, dtype=torch.float32) * torch.tensor(T, dtype=torch.float32))
    survival_probs = torch.exp(-lambda_0 * time_grid[-1])
    cva = LGD * torch.mean((1 - survival_probs) * discount_factors * payoffs)
    return cva.item()

# Simulate stock prices and default times
# Simulate stock prices
S = simulate_stock_paths(S0, r, sigma, T, time_steps, n_paths)

# Calculate CVA for lambda_0 + epsilon
default_times_plus = simulate_default_times(lambda_0 + epsilon, time_steps, T, n_paths)
cva_plus = calculate_cva(LGD, S, K, T, r, lambda_0 + epsilon, default_times_plus, time_grid)

# Calculate CVA for lambda_0 - epsilon
default_times_minus = simulate_default_times(lambda_0 - epsilon, time_steps, T, n_paths)
cva_minus = calculate_cva(LGD, S, K, T, r, lambda_0 - epsilon, default_times_minus, time_grid)

# Delta of CVA with respect to lambda_0 using central difference method
delta_cva = (cva_plus - cva_minus) / (2 * epsilon)

print(f"CVA (lambda_0 + epsilon): {cva_plus:.5f}")
print(f"CVA (lambda_0 - epsilon): {cva_minus:.5f}")
print(f"Delta of CVA w.r.t lambda_0: {delta_cva:.5f}")


In [ ]:
import torch

# Parameters for the European call option
S0 = 100.0  # Initial stock price
K = 90.0  # Strike price
T = 2.0  # Time to maturity (2 years)
r = 0.01  # Risk-free rate
sigma = 0.25  # Volatility

# CVA and intensity model parameters
LGD = 0.6  # Loss given default
lambda_0 = 1.0  # Initial hazard rate
n_paths = 5000  # Number of Monte Carlo paths
time_steps = 200  # Number of time steps

# Generate time grid
dt = T / time_steps
time_grid = torch.linspace(0, T, time_steps)

# Simulate stock price paths using Geometric Brownian Motion (GBM)
def simulate_stock_paths(S0, r, sigma, T, time_steps, n_paths):
    dt = T / time_steps
    # Convert dt to a PyTorch tensor
    dt = torch.tensor(dt, dtype=torch.float32) # This line converts dt to a tensor
    Z = torch.randn(n_paths, time_steps)  # Random shocks
    increments = (r - 0.5 * sigma**2) * dt + sigma * torch.sqrt(dt) * Z
    increments = torch.cat([torch.zeros(n_paths, 1), increments], dim=1)
    log_S = torch.cumsum(increments, dim=1)
    S = S0 * torch.exp(log_S)
    return S

# Simulate default times using an intensity model
def simulate_default_times(lambda_t, time_steps, T, n_paths):
    dt = T / time_steps
    cumulative_hazard = torch.cumsum(lambda_t * dt * torch.ones(time_steps), dim=0)
    survival_probs = torch.exp(-cumulative_hazard)
    defaults = torch.rand(n_paths, time_steps) > survival_probs
    default_times = torch.argmax(defaults.type(torch.int), dim=1) * dt
    default_times[default_times == 0] = T + 1  # No default
    return default_times

# Calculate CVA
def calculate_cva(LGD, S, K, T, r, lambda_t, time_grid):
    payoffs = torch.maximum(S[:, -1] - K, torch.tensor(0.0))  # Max(S_T - K, 0)
    # Convert r and T to tensors before using them in torch.exp
    discount_factors = torch.exp(-torch.tensor(r, dtype=torch.float32) * torch.tensor(T, dtype=torch.float32))
    cumulative_hazard = torch.sum(lambda_t * dt * torch.ones(time_steps))
    survival_probs = torch.exp(-cumulative_hazard)
    cva = LGD * torch.mean((1 - survival_probs) * discount_factors * payoffs)
    return cva

# Enable automatic differentiation for lambda_0
lambda_t = torch.tensor(lambda_0, requires_grad=True)

# Simulate stock prices
S = simulate_stock_paths(S0, r, sigma, T, time_steps, n_paths)

# Compute CVA using automatic differentiation
cva = calculate_cva(LGD, S, K, T, r, lambda_t, time_grid)

# Perform backpropagation to compute the gradient of CVA w.r.t. lambda_0
cva.backward()

# Extract the gradient (delta of CVA with respect to lambda_0)
delta_cva = lambda_t.grad.item()

print(f"CVA: {cva.item():.5f}")
print(f"Delta of CVA w.r.t lambda_0: {delta_cva:.5f}")